In [2]:
import pandas as pd

In [7]:
df = pd.read_csv('NonfictionCirc.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   BrowseAuthor                1798 non-null   object 
 1   BrowseTitle                 1974 non-null   object 
 2   PublicationYear             1972 non-null   float64
 3   CallNumber                  1974 non-null   object 
 4   TotalItemLifetimeCircCount  1974 non-null   int64  
 5   TotalItemYTDCircCount       1974 non-null   int64  
 6   Unnamed: 6                  0 non-null      float64
 7   Call No. Color Code         11 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 123.5+ KB
None
